# Simulating DSA-200 Images

From tyrones_stuff, simulate a very large image, e.g.:

`python sky_model.py --first ~/catalogs/first/first_14dec17.fits -o test_first.fits`

TODO: add more sources and make larger area. be sure to set npix to produce correct pixel scale

The simulation process will apply image-plane modifications as they are applied by propagation and measurement (going "forward").
1. apply ionospheric screen
2. apply primary beam and cut out 16k x 16k field (~10 sq deg)
3. add noise
4. convolve with PSF

Then iterate over new screens and pointing directions to create a sequence of measurements of a "tile" (e.g., 5x5).

In [1]:
input_image = '/home/claw/code/tyrones_stuff/test_first.fits'  # toy image

In [2]:
from data import fits_image
image, wcs = fits_image(input_image)

2024-09-10 17:09:37.202049: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.1 which is older than the ptxas CUDA version (12.6.68). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


## 1. Ionosphere

In [3]:
# create difference array
#diff = ref_xy - sources_xy  # offset of x/y in units of pixels
#dxmodel = RBFInterpolator(sources_xy, diff[:, 0], kernel='linear', smoothing=smoothing, neighbors=neighbors)
#dymodel = RBFInterpolator(sources_xy, diff[:, 1], kernel='linear', smoothing=smoothing, neighbors=neighbors)

#offsets = compute_offsets(dxmodel, dymodel)  # IMAGE_SIZE^2 x 2
#image_indices = np.indices((IMAGE_SIZE, IMAGE_SIZE)).swapaxes(0, 2)[:, :, ::-1].reshape((IMAGE_SIZE * IMAGE_SIZE, 2))
#interp = CloughTocher2DInterpolator(image_indices - offsets, np.ravel(image_data))
#dewarped = compute_interpolation(interp)

# "dewarped" can also be "warped"!
#
# compute_offsets and compute_interpolation are in https://github.com/ovro-lwa/image-plane-correction/blob/main/scripts/fits_warp_lwa.py

## 2. Primary beam and cut out

In [4]:
from util import indices, rescale_quantile, gaussian_filter, match_histograms
from interactive import toggle_images, plot_image

In [5]:
img1 = gaussian_filter(image, sigma=3)
img1r = rescale_quantile(img1, 0.5, 0.95)

E0910 17:09:43.935164 3887413 cuda_blas.cc:202] failed to create cublas handle: the library was not initialized
E0910 17:09:43.935232 3887413 cuda_blas.cc:205] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2024-09-10 17:09:44.008918: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:802] None of the algorithms provided by cuDNN heuristics worked; trying fallback algorithms.
2024-09-10 17:09:44.008984: W external/xla/xla/service/gpu/conv_algorithm_picker.cc:805] Conv: (f32[8192,1,8192]{2,1,0}, u8[0]{0}) custom-call(f32[8192,1,8192]{2,1,0}, f32[1,1,25]{2,1,0}), window={size=25 pad=12_12}, dim_labels=bf0_oi0->bf0, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_res

XlaRuntimeError: UNKNOWN: Failed to determine best cudnn convolution algorithm for:
%cudnn-conv = (f32[8192,1,8192]{2,1,0}, u8[0]{0}) custom-call(f32[8192,1,8192]{2,1,0} %Arg_0.1, f32[1,1,25]{2,1,0} %Arg_1.2), window={size=25 pad=12_12}, dim_labels=bf0_oi0->bf0, custom_call_target="__cudnn$convForward", metadata={op_name="jit(conv_general_dilated)/jit(main)/conv_general_dilated[window_strides=(1,) padding=((12, 12),) lhs_dilation=(1,) rhs_dilation=(1,) dimension_numbers=ConvDimensionNumbers(lhs_spec=(0, 1, 2), rhs_spec=(0, 1, 2), out_spec=(0, 1, 2)) feature_group_count=1 batch_group_count=1 precision=None preferred_element_type=None]" source_file="/tmp/ipykernel_3887413/1182500175.py" source_line=1}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}

Original error: INTERNAL: All algorithms tried for (f32[8192,1,819

In [7]:
plot_image(img1r)

NameError: name 'img1r' is not defined

## 3. Add noise

In [6]:
# just add some low-spatial-freq rumble?

## 4. Convolve with PSF

In [ ]:
# TODO: find best PSF
#input_psf = '/fastpool/data/W-GC-8000chan-briggs09-big-psf.fits'

In [ ]:
#psf_image, psf_wcs = fits_image(psf_filename)
#convolved = convolve(img1r, psf_image, mode="same", method="fft")